In [1]:
import numpy as np
import sys
import os
sys.path.append(os.path.abspath('../'))
from scipy.io import loadmat
import loaders
from preprocessing.vsdi_preprocessing import clean_outliers,pca_ica,glm
from pathlib import Path

In [2]:
datapath = Path('/ceph/imaging1/davide/ATC_Data_preprocessed')
atc = loadmat(datapath.joinpath('A04/Day1/ATC1.mat'))

In [3]:
b_data = loaders.extract_behavioural_data(atc)

In [ ]:
def make_design_matrix(b_data,fps=50):
    # Design matrix
    # Returns a matrix of size (time, 6) with the following columns:
    # 0: CS+ (2s)
    # 1: CS+ trace (1s)
    # 2: CS- (2s)
    # 3: CS- trace (1s)
    # 4: Reward (1s)
    # 5: Lick


    Lick = b_data['Lick']
    CSp = b_data['CSp']
    CSn = b_data['CSn']
    frames = b_data['frames']

    length = len(frames)
    X = np.zeros((length, 6))

    # Iterate over lick events
    for i in range(len(Lick)):
        frame = np.argmin(np.abs(frames - Lick[i]))
        X[frame:frame+4, 5] = 1

    # Iterate over CSp events
    for i in range(len(CSp)):
        frame = np.argmin(np.abs(frames - CSp[i]))
        X[frame:frame + (fps * 2), 0] = 1
        frame += (fps * 2) + 1
        X[frame:frame + (fps * 1), 1] = 1
        frame += (fps * 1) + 1
        X[frame:frame + (fps * 1), 4] = 1

    # Iterate over CSn events
    for i in range(len(CSn)):
        frame = np.argmin(np.abs(frames - CSn[i]))
        X[frame:frame + (fps * 2), 2] = 1
        frame += (fps * 2) + 1
        X[frame:frame + (fps * 1), 3] = 1

    return X

In [ ]:
# LOAD TIMECOURSES
# SPLIT EACH DAY USING ATC1 info
# FIT GLM ON EACH COMPONENT ON EACH DAY
# SAVE RESULTS

In [ ]:
def design_matrix(b_data):
    # Design matrix
    # Returns a matrix of size (time, 6) with the following columns:
    # 0: CS+ (2s)
    # 1: CS+ trace (1s)
    # 2: CS- (2s)
    # 3: CS- trace (1s)
    # 4: Reward (1s)
    # 5: Lick

    fps = 50

    Lick = b_data['Lick']
    CSp = b_data['CSp']
    CSn = b_data['CSn']
    frames = b_data['frames']

    length = len(frames)
    X = np.zeros((length, 6))

    # Iterate over lick events
    for i in range(len(Lick)):
        frame = np.argmin(np.abs(frames - Lick[i]))
        X[frame:frame+4, 5] = 1

    # Iterate over CSp events
    for i in range(len(CSp)):
        frame = np.argmin(np.abs(frames - CSp[i]))
        X[frame:frame + (fps * 2), 0] = 1
        frame += (fps * 2) + 1
        X[frame:frame + (fps * 1), 1] = 1
        frame += (fps * 1) + 1
        X[frame:frame + (fps * 1), 4] = 1

    # Iterate over CSn events
    for i in range(len(CSn)):
        frame = np.argmin(np.abs(frames - CSn[i]))
        X[frame:frame + (fps * 2), 2] = 1
        frame += (fps * 2) + 1
        X[frame:frame + (fps * 1), 3] = 1

    return X